**Imports**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pickle
import time
import missingno as msno
from statistics import mean
from collections import Counter, OrderedDict
from IPython.display import display
import import_ipynb
import UTILS as utils

import FundamentalAnalysis as fa

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
api_key = "Here would go my personal API key"

------------

**Load data**

*Files with data from year with displacement -X*

In [167]:
file1 = 'DATA_ttm/S_P500_data_ttm_Xyeardisp/S_P500_dataframe_with_ratios_ttm_Xyeardisp.csv'
file2 = 'DATA_ttm/S_P500_data_ttm_Xyeardisp/S_P500_companies_financials_data_ttm_Xyeardisp.pickle'

In [168]:
data_loaded = utils.data_loading(file1, file2)

In [169]:
df = data_loaded[0]
dict_companies = data_loaded[1]

In [170]:
df.head(2)

,_PAT_margin,_EBITDA_margin,_return_on_equity,_return_on_assets,_return_on_capital_employed,_gross_margin,_operating_margin,_operating_cash_flow_ratio,_return_on_invested_capital,_return_on_sales,_fixed_asset_turnover,_working_capital_turnover,_total_asset_turnover,_inventory_turnover,_days_of_inventory_on_hand,_accounts_receivables_turnover,_payables_turnover,_debt_to_assets,_interest_coverage,_asset_to_equity,_debt_to_equity,_equity_multiplier,_total_debt_to_capitalization,_total_debt_to_capital,_net_debt_to_EBITDA,_degree_financial_leverage,_earnings_per_share,_book_value_per_share,_cash_earnings_per_share,_price_to_earnings,_price_to_book_value,_price_to_sales,_price_to_free_cash_flow,_price_earnings_to_growth,_dividend_yield,_dividend_payout,_enterprise_value_to_ebitda,_enterprise_value_to_sales,_current_ratio,_quick_ratio,_cash_ratio,_days_of_sales_outstanding,_days_of_sales_in_inventory,_days_of_payables_outstanding,_operating_cycle,_cash_conversion_cycle
ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MMM,0.167402,0.271661,0.102917,0.031030,0.047709,0.468315,0.208277,2.300579,0.047709,0.208277,0.866823,5.229256,0.185262,0.978114,92.077553,1.813772,1.642398,0.696076,15.093071,3.294063,1.176688,3.309888,0.566805,0.566805,5.917859,2.226105,2.555000,24.922730,3.219128,18.235344,7.483685,12.189879,73.837823,-2.09291,0.007957,0.580389,50.806934,3.449114,1.743940,1.115540,0.515781,49.624411,201.254556,54.859324,141.701964,86.842640
AOS,0.137160,0.199539,0.065875,0.037287,0.066318,0.370851,0.171587,21.825725,0.066318,0.171587,1.483506,5.073510,0.271250,1.547255,58.337771,1.472644,0.873009,0.431103,143.894444,1.761376,0.082905,1.761376,0.079406,0.079406,-1.930628,1.033130,0.757969,11.493758,1.000958,23.926370,6.235913,13.052287,98.040415,1.01074,0.003732,0.355069,63.643361,3.159238,1.736831,1.302403,0.518312,61.207901,248.232043,103.114310,119.545672,16.431362


----------------

**Add sectors and industries information to the dataset**

In [171]:
sectors_df, industries_df = utils.get_company_sectors_and_industries(df, dict_companies)

In [172]:
if set(['sector','industry']).issubset(df.columns) == False:
    df.insert(0, "sector", sectors_df.sector)
    df.insert(1, "industry", industries_df.industry)

-------------------

**Compute intrinsic value**

In [31]:
def compute_debt_component(dict_companies, ticker, disp):
    
    #Compute cost of debt
    income_statement = dict_companies[ticker]['_Company__income_statement']
    interest_expense = income_statement.loc['interestExpense'][4*disp:4+(4*disp)].sum()

    balance_sheet = dict_companies[ticker]['_Company__balance_sheet']
    total_debt = balance_sheet.loc['totalDebt'][4*disp:4+(4*disp)].mean()
        
    if interest_expense==0 or total_debt==0:
        average_cost_of_debt = 0
    else:  
        average_cost_of_debt = interest_expense/total_debt
    
    income_before_tax = income_statement.loc['incomeBeforeTax'][4*disp:4+(4*disp)].sum()
    income_tax_expense = income_statement.loc['incomeTaxExpense'][4*disp:4+(4*disp)].sum()
    
    if income_tax_expense==0 or income_before_tax==0:
        effective_tax_rate = 0
    else:
        effective_tax_rate = round(income_tax_expense/income_before_tax,4)
    
    debt_component = average_cost_of_debt*(1-effective_tax_rate)

    return debt_component

In [32]:
def compute_equity_component(dict_companies, ticker, average_market_return, date):
    
    #Compute cost of equity
    
    #Using capital asset pricing model

    ten_year_bond_rate = pd.read_csv(filepath_or_buffer='10-year-treasury-bond-rate-yield-chart.csv', index_col=0)
    risk_free_rate = ten_year_bond_rate.loc[ten_year_bond_rate.index.str.contains(date)].iloc[0].value / 100

    beta = dict_companies[ticker]['_Company__profile'].loc['beta'][0]

    equity_component = risk_free_rate+beta*(average_market_return-risk_free_rate)
    
    return equity_component

In [33]:
def compute_WACC(dict_companies, ticker, average_market_return, date, disp):
    
    #Compute debt component
    debt_component = compute_debt_component(dict_companies, ticker, disp)
    
    #Compute equity component
    equity_component = compute_equity_component(dict_companies, ticker, average_market_return, date)
    
    
    #Get the weights for debt and equity

    enterprise_value = dict_companies[ticker]['_Company__enterprise_value']
    balance_sheet = dict_companies[ticker]['_Company__balance_sheet']
    
    market_cap = enterprise_value.loc['marketCapitalization'][4*disp:4+(4*disp)].mean()
    total_debt = balance_sheet.loc['totalDebt'][4*disp:4+(4*disp)].mean()
    total_debt_cap = total_debt + market_cap
    w_debt = total_debt/total_debt_cap
    w_equity = market_cap/total_debt_cap
    
    #Compute WACC by combining debt and equity components and weights
    WACC = (w_debt*debt_component)+(w_equity*equity_component)
    
    return WACC

In [51]:
def compute_intrinsic_value_dcf(dict_companies, ticker, average_market_return, year_disp=0, discount_rate=None, 
                                terminal_growth_rate=0.025, years=5):
    
    # According to https://stablebread.com/how-to-value-a-company-using-the-discounted-cash-flow-model/,
    #f you're unsure on how many years to forecast FCF, stick to 5 years. If you decide on projecting FCF 10 years (or even longer)
    #just know that the further these numbers are projected out, the more these later periods are subject to estimation error
    
    #The terminal growth rate is the constant rate at which a firm’s expected free cash flows are assumed to grow indefinitely
    #According to https://corporatefinanceinstitute.com/resources/knowledge/valuation/what-is-terminal-growth-rate/,
    #the terminal growth rates typically range between the historical inflation rate (2%-3%) and the average GDP growth rate (3%-4%) at this stage.
    
    
    #We'll work with num years free cash flow, but to compute the fcf_growth_rate we will also need the -(num+1) year
    cash_flow_statement = dict_companies[ticker]['_Company__cash_flow_statement']
    
    try:
        date = cash_flow_statement.columns[4*year_disp]
        current_number_of_shares = dict_companies[ticker]['_Company__enterprise_value'].loc['numberOfShares'][4*year_disp]
    except:
        return "Error. No data for this ticker"
    
    #This is used to check if any of the tickers contains a cf statement with less than the required columns (like CEG)
    if len(cash_flow_statement.columns) < ((4*year_disp)+((years+1)*4)):
        return "Can't compute the intrinsic value for that amount of years for ticker: "+str(ticker)
    
    #If discount rate is not explicitly specified, then we have to compute it using the WACC
    if discount_rate == None:
        discount_rate = compute_WACC(dict_companies, ticker, average_market_return, date, year_disp)
           
    numplusoney_cash_flow_statement = cash_flow_statement[cash_flow_statement.columns[4*year_disp:(4*year_disp)+((years+1)*4)]]
    pastnumplusoney_fcf = []                                           
    for i in range(len(numplusoney_cash_flow_statement.columns)):
        if i%4 == 0: pastnumplusoney_fcf.insert(0, numplusoney_cash_flow_statement.\
                     loc['freeCashFlow', numplusoney_cash_flow_statement.columns[i:i+4]].sum())

    #Compute past num average fcf growth rate
    pastnumy_fcf_growth_rate = []
    for i in range(len(pastnumplusoney_fcf)-1):
        if pastnumplusoney_fcf[i] == 0:
            pastnumy_fcf_growth_rate.append(0)
        else:
            pastnumy_fcf_growth_rate.append((pastnumplusoney_fcf[i+1]-pastnumplusoney_fcf[i])/pastnumplusoney_fcf[i])
    
    avg_fcf_growth_rate = mean(pastnumy_fcf_growth_rate)

    #Delete the -(num+1) fcf
    pastnumy_fcf = pastnumplusoney_fcf[1:]
    
    
    future_fcf = []
    discount_factor = []
    discounted_future_fcf = []
    
    
    #Compute terminal value: determines a company's value into perpetuity beyond a set forecast period—usually five years.
    #Terminal value assumes a business will grow at a set growth rate forever after the forecast period.
    terminal_value = pastnumy_fcf[-1]*(1+terminal_growth_rate)/(discount_rate)
    
    
    for year in range(1, years+1):
        cash_flow = pastnumy_fcf[-1]*(1+avg_fcf_growth_rate)**year
        future_fcf.append(cash_flow)
        discount_factor.append((1+discount_rate)**year)
        
    #Compute discounted free cash flows
    for i in range(years):
        discounted_future_fcf.append(future_fcf[i]/discount_factor[i])
    discounted_terminal_value = terminal_value/(1+discount_rate)**years
    discounted_future_fcf.append(discounted_terminal_value)
    
    current_value = sum(discounted_future_fcf)
    
    current_intrinsic_value = round(current_value/current_number_of_shares, 2)

    
    return current_intrinsic_value, date

In [52]:
def stock_price_intrinsic_value_comparator(dict_companies, ticker, intrinsic_value, date):
    
    stock_data = fa.stock_data(ticker, interval="1d")
    stock_data.index = pd.to_datetime(stock_data.index)
    
    format_date = pd.to_datetime(date+'-01')
    stock_price = stock_data.iloc[stock_data.index.get_loc(format_date, method='nearest')].close
        
    if stock_price == 0:
        pr_intrinsic_price = 0
    else:
        pr_intrinsic_price = intrinsic_value/stock_price
    
    return pr_intrinsic_price

In [ ]:
dcf_by_sector = dict()
sectors = utils.get_sectors_and_industries(df, dict_companies)[0]
for sector in sectors:
    sector_df = utils.get_sector_companies(df, dict_companies, sector)
    dcf_by_sector[sector] = dict()
    for ticker in sector_df.index:
        #We'll take the average return of the market from the average return of S&P500 which is 10,5% but being conservative, we'll leave it at 10%
        intrinsic = compute_intrinsic_value_dcf(dict_companies, ticker, average_market_return=0.1, year_disp=(X years of disp))
        if isinstance(intrinsic, str):
            dcf_by_sector[sector][ticker] = [np.nan]
            dcf_by_sector[sector][ticker].append(np.nan)
        else:
            dcf_by_sector[sector][ticker] = [intrinsic[0]]
            dcf_by_sector[sector][ticker].append(stock_price_intrinsic_value_comparator(dict_companies, ticker, intrinsic[0], intrinsic[1]))

In [ ]:
dcf_by_sector.keys()

In [ ]:
with open(f'models_outputs/DCF_output/X_year_disp/dcf_by_sector_Xyear.pickle', 'wb') as file:
    pickle.dump(dcf_by_sector, file)

**Retrieve the saved dictionary**

In [27]:
dcf_by_sector_dict = dict()
with open('models_outputs/DCF_output/X_year_disp/dcf_by_sector_Xyear.pickle','rb') as file:
    raw_data = file.read()
    dcf_by_sector_dict.update(pickle.loads(raw_data))

In [28]:
dcf_by_sector = dict()
for sector in dcf_by_sector_dict.keys():
    dcf_by_sector[sector] = pd.DataFrame.from_dict(dcf_by_sector_dict[sector], orient='index', columns=['intrinsic_value', 'intrval_vs_currentprice'])

In [29]:
dcf_by_sector['Industrials']

,intrinsic_value,intrval_vs_currentprice
MMM,123.72,0.831005
AOS,38.24,0.557597
ADP,82.19,0.402025
ALK,51.98,0.925900
ALLE,52.82,0.408760
AAL,427.89,23.824611
AME,43.81,0.328953
AVY,152.49,0.742333
BA,-125.32,-0.654413
CHRW,2.41,0.022392
